#### What are you trying to do in this notebook?
We recognize the works of artists through their unique style, such as color choices or brush strokes. Artists like Claude Monet can now be imitated with algorithms thanks to generative adversarial networks (GANs). In this competition, I will bring that style to my photos or recreate the style from scratch!

Computer vision has advanced tremendously in recent years and GANs are now capable of mimicking objects in a very convincing way. But creating museum-worthy masterpieces is thought of to be, well, more art than science. So can (data) science, in the form of GANs, trick classifiers into believing I’ve created a true Monet? That’s the challenge I’ll take on!

#### Why are you trying it?
A GAN consists of at least two neural networks: a generator model and a discriminator model. The generator is a neural network that creates the images. For this competition, I generate images in the style of Monet. This generator is trained using a discriminator.
The two models will work against each other, with the generator trying to trick the discriminator, and the discriminator trying to accurately classify the real vs. generated images.

My task is to build a GAN that generates 7,000 to 10,000 Monet-style images.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install wandb
import wandb
# Wandb Login
wandb.login()
# wandb config
#WANDB_CONFIG = {'competition': 'Sartorius', '_wandb_kernel': 'neuracort', 'entity':"kohyun1207"}
run = wandb.init(project='Monet Cycle GAN')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
from PIL import Image
import itertools
from tqdm import tqdm


import torch
import torchvision


from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import torch.nn as nn
import torch.optim.lr_scheduler as lr_scheduler

In [ ]:
path = ['../input/gan-getting-started/monet_jpg/','../input/gan-getting-started/photo_jpg/']
monet = os.listdir('../input/gan-getting-started/monet_jpg')
photo = os.listdir('../input/gan-getting-started/photo_jpg')

In [ ]:
#visulization
def visulization(x,y):
    plt.figure(figsize = (10,5))
    plt.subplot(1,2,1)
    plt.title('actual')
    plt.imshow(x.cpu().detach().numpy().transpose(1,2,0))
    plt.tick_params(left = False, bottom = False, labelleft = False, labelbottom = False)
    plt.subplot(1,2,2)
    plt.title('fake')
    plt.imshow(y.cpu().detach().numpy().transpose(1,2,0))
    plt.tick_params(left = False, bottom = False, labelleft = False, labelbottom = False)
    plt.show()
    
# weight initialization 
def weight_init(m : 'model'):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('Instance') != -1:
        nn.init.normal_(m.weight.data, 1.0, 0.02)
        nn.init.constant_(m.bias.data, 0)
#Nomalizer 
class normalize(object):
    def __call__(self, inputs):
        mean = 0.5 
        std = 0.5
        inputs = ((inputs - mean) / std)
        return inputs

def update_req_grad(models, requires_grad=True):
    for model in models:
        for param in model.parameters():
            param.requires_grad = requires_grad

In [ ]:
#Defining Dataset

train_transform = T.Compose([T.RandomHorizontalFlip(p = 0.5),
                             T.RandomVerticalFlip(p = 0.5),
                             T.RandomRotation(180),
                             T.ToTensor(),
                             normalize()])

class CustomDataset(Dataset):
    def __init__(self, path, monet, photo, transforms = None, seed = 777):
        self.path = path
        self.monet = monet
        self.photo = photo
        self.seed = seed
        self.transforms = transforms
        self.photo_len = len(self.monet)
        self.monet_len = len(self.photo)
        self.length_dataset = max(self.photo_len, self.monet_len)
        
        
    def __len__(self):
        return len(self.photo)
    
    def __getitem__(self, idx):
        
        
        #get path
        monet_path = self.path[0] + self.monet[idx % 300]
        photo_path = self.path[1] + self.photo[idx]
        #get image
        monet = Image.open(monet_path).convert('RGB')
        photo = Image.open(photo_path).convert('RGB')
        #image Transform
        if self.transforms:
            torch.manual_seed(self.seed)
            monet = self.transforms(monet)
        if self.transforms:
            torch.manual_seed(self.seed)
            photo = self.transforms(photo)
        return monet, photo

In [ ]:
#Cycle GAN
#Generator(Photo <--> Monet)
#Defining layers
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        
        #1. Conv layers
        
        def CLayer(in_ch, out_ch, kernel_size = 3, stride = 1, padding = 1, bias = True, norm = 'bnorm', relu = 'relu'):
            layers = []

            layers += [nn.ReflectionPad2d(padding)]
                         
            layers += [nn.Conv2d(in_channels = in_ch,
                                 out_channels = out_ch,
                                 kernel_size = kernel_size,
                                 stride = stride,
                                 padding = 0,
                                 bias = bias)]
            if not norm is None:
                if norm == 'bnorm':
                    layers += [nn.BatchNorm2d(num_features = out_ch)]
                elif norm =='inorm':
                    layers += [nn.InstanceNorm2d(num_features = out_ch)]
            
            if relu == 'relu':
                layers += [nn.ReLU()]
            elif relu == 'leakyrelu':
                layers += [nn.LeakyReLU()]
                
            return nn.Sequential(*layers)
        
        #2. Residual Blocks
        
        def Rblock(in_ch, out_ch, kernel_size = 3, stride = 1, padding = 1, bias = True, norm = 'bnorm', relu = 0.0):
            layers = []
            layers += [CLayer(in_ch = in_ch,
                              out_ch = out_ch,
                              kernel_size = kernel_size,
                              stride = stride,
                              padding = padding,
                              bias = bias,
                              norm = norm)]
            
            layers += [CLayer(in_ch = in_ch,
                              out_ch = out_ch,
                              kernel_size = kernel_size,
                              stride = stride,
                              padding = padding,
                              bias = bias,
                              norm = norm,
                             relu = None)]
            return nn.Sequential(*layers)
        
        #3. Transpose Conv layers
        
        def TCLayer(in_ch, out_ch, kernel_size = 3, stride = 1, padding = 1, output_padding = 1, bias = True, norm = 'bnorm', relu = 'relu'):
            layers = []
            layers += [nn.ConvTranspose2d(in_channels = in_ch,
                                         out_channels = out_ch,
                                         kernel_size = kernel_size,
                                         stride = stride,
                                         padding = padding,
                                         output_padding = output_padding,
                                         bias = bias)]
            if not norm is None:
                if norm == 'bnorm':
                    layers += [nn.BatchNorm2d(num_features = out_ch)]
                elif norm =='inorm':
                    layers += [nn.InstanceNorm2d(num_features = out_ch)]
                    
            if relu == 'relu':
                layers += [nn.ReLU()]
            elif relu == 'leakyrelu':
                layers += [nn.LeakyReLU(0.2)]
                
            return nn.Sequential(*layers)
        
        #Encoder
        self.encoder1 = CLayer(in_ch = 3, out_ch = 64 , kernel_size = 7, stride = 1, padding = 3, norm = 'inorm', relu = 'relu')
        self.encoder2 = CLayer(in_ch = 64, out_ch = 128 , kernel_size = 3, stride = 2, padding = 1, norm = 'inorm', relu = 'relu')
        self.encoder3 = CLayer(in_ch = 128, out_ch = 256 , kernel_size = 3, stride = 2, padding = 1, norm = 'inorm', relu = 'relu')

        #Transformer
        res_layer = []
        
        for i in range(6):
            res_layer += [Rblock(in_ch = 256, out_ch = 256, kernel_size = 3, stride = 1, padding = 1, norm = 'inorm', relu = 'relu')]
            
        self.trans = nn.Sequential(*res_layer)

        #Decoder
        self.decoder1 = TCLayer(in_ch = 256, out_ch = 128, kernel_size = 3, stride = 2, padding = 1, output_padding = 1, norm = 'inorm', relu = 'relu')
        self.decoder2 = TCLayer(in_ch = 128, out_ch = 64, kernel_size = 3, stride = 2, padding = 1, output_padding = 1, norm = 'inorm', relu = 'relu')
        self.decoder3 = CLayer(in_ch = 64, out_ch = 3 , kernel_size = 7, stride = 1, padding = 3, norm = 'inorm', relu = 'relu')
        
    def forward(self, input):
        
        #Encoder
        x = self.encoder1(input)
        x = self.encoder2(x)
        x = self.encoder3(x)
        
        #Transformer
        x = self.trans(x)
        
        #Decoder
        x = self.decoder1(x)
        x = self.decoder2(x)
        x = self.decoder3(x)
        
        output = torch.tanh(x)
              
        return output

In [ ]:
#Cycle GAN
#Discriminator(Photo <--> Monet)
#Defining layers
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        
        #1. Conv layers
        
        def CLayer(in_ch, out_ch, kernel_size = 3, stride = 1, padding = 1, bias = True, norm = 'bnorm', relu = 'relu'):
            layers = []

            layers += [nn.ReflectionPad2d(padding)]
                         
            layers += [nn.Conv2d(in_channels = in_ch,
                                 out_channels = out_ch,
                                 kernel_size = kernel_size,
                                 stride = stride,
                                 padding = 0,
                                 bias = bias)]
            if not norm is None:
                if norm == 'bnorm':
                    layers += [nn.BatchNorm2d(num_features = out_ch)]
                elif norm =='inorm':
                    layers += [nn.InstanceNorm2d(num_features = out_ch)]
            
            if relu == 'relu':
                layers += [nn.ReLU()]
            elif relu == 'leakyrelu':
                layers += [nn.LeakyReLU(0.2)]
                
            return nn.Sequential(*layers)
        
        self.decoder1 = CLayer(in_ch = 3, out_ch = 64, kernel_size = 4, stride = 2, padding = 1, bias = False, norm = 'bnorm', relu = 'leakyrelu')
        self.decoder2 = CLayer(in_ch = 64, out_ch = 128, kernel_size = 4, stride = 2, padding = 1, bias = False, norm = 'bnorm', relu = 'leakyrelu')
        self.decoder3 = CLayer(in_ch = 128, out_ch = 256, kernel_size = 4, stride = 2, padding = 1, bias = False, norm = 'bnorm', relu = 'leakyrelu')
        self.decoder4 = CLayer(in_ch = 256, out_ch = 512, kernel_size = 4, stride = 1, padding = 1, bias = False, norm = 'bnorm', relu = 'leakyrelu')
        self.decoder5 = CLayer(in_ch = 512, out_ch = 1, kernel_size = 4, stride = 1, padding = 1, bias = False, norm = None, relu = None)
        
    def forward(self, input):
        
        x = self.decoder1(input)
        x = self.decoder2(x)
        x = self.decoder3(x)
        x = self.decoder4(x)
        x = self.decoder5(x)
        
        output = torch.sigmoid(x)
        
        return output

In [ ]:
#Training
#configure
lr = 0.001
n_epoch = 10
batch_size = 2
setting_patience = 7



device = 'cuda' if torch.cuda.is_available() else 'cpu'
#Defineing Model
GM2P = Generator().to(device) #Monet ---> Photo
GP2M = Generator().to(device) #Photo ---> Monet
DM2P = Discriminator().to(device) #Monet ---> Photo
DP2M = Discriminator().to(device) #Photo ---> Monet

#Weight Initialize
weight_init(GM2P)
weight_init(GP2M)
weight_init(DM2P)
weight_init(DP2M)

#Loss Functions
#1. GAN loss - L2
gan_loss = nn.BCELoss().to(device)
#2. Cycle loss - L1
cyc_loss = nn.L1Loss().to(device)
#3. Identity loss - L1
iden_loss = nn.L1Loss().to(device)

#Optimizer (Connet Monet <---> Photo)
OptimG = torch.optim.Adam(itertools.chain(GM2P.parameters(), GP2M.parameters()), lr = lr, betas=(0.5, 0.999))
OptimD = torch.optim.Adam(itertools.chain(DM2P.parameters(), DP2M.parameters()), lr = lr, betas=(0.5, 0.999))

schedulerG = lr_scheduler.LambdaLR(optimizer=OptimG,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)
schedulerD = lr_scheduler.LambdaLR(optimizer=OptimD,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

#Get Data from Dataloader
train_dataset = CustomDataset(path, monet, photo, transforms = train_transform, seed = 777)
train_dataloader = DataLoader(train_dataset, batch_size = batch_size, shuffle  = True, num_workers = 2)
train_total_batch = len(train_dataloader)
loss_G_list = []
loss_D_list = []

In [ ]:
#Training & Evaluation

best_G_loss = 100
best_D_loss = 100

total_patience = 0
for epoch in range(n_epoch):
    
    
    GM2P.train()
    GP2M.train()
    DM2P.train()
    DP2M.train()

    loss_G_avg = 0.0
    loss_D_avg = 0.0
    
    with tqdm(train_dataloader, unit = 'batch') as train_bar:
        
        for monets, photos in train_bar:  
            
            torch.cuda.empty_cache()
            monets = monets.float().to(device)
            photos = photos.float().to(device)
            
            #forward Generator
            update_req_grad([DM2P, DP2M], False)
            OptimG.zero_grad()
            
            fake_photo = GM2P(monets)
            fake_monet = GP2M(photos)

            cycl_monet = GP2M(fake_photo)
            cycl_photo = GM2P(fake_monet)
            
            ident_monet = GP2M(monets)
            ident_photo = GM2P(photos)
            
            
            #Caculating loss (Identity, Advrsarial, cycle consistency)
            #identity loss
            ident_loss_monet = iden_loss(ident_monet, monets) * 10 * 0.5
            ident_loss_photo = iden_loss(ident_photo, photos) * 10 * 0.5
            #Cycle loss
            cycle_loss_monet = cyc_loss(cycl_monet, monets) * 10
            cycle_loss_photo = cyc_loss(cycl_photo, photos) * 10
            #Adversarial loss
            pred_fake_monet = DM2P(fake_monet.detach())
            pred_fake_photo = DP2M(fake_photo.detach())    
            
            adv_loss_monet = gan_loss(pred_fake_monet, torch.ones_like(pred_fake_monet))
            adv_loss_photo = gan_loss(pred_fake_photo, torch.ones_like(pred_fake_photo))
            
            #Generater Loss
            loss_G = (ident_loss_monet + ident_loss_photo ) + (cycle_loss_monet + cycle_loss_photo) + (adv_loss_monet + adv_loss_photo)
            loss_G_avg += loss_G.item() / train_total_batch
            #Generator Backward
    
            loss_G.backward(retain_graph=True)
            OptimG.step()
            
            #forward Discriminator
            update_req_grad([DM2P, DP2M], True)
            OptimD.zero_grad()
            
            pred_real_monet = DP2M(photos)
            pred_real_photo = DM2P(monets)
            
            #Discriminator loss
            loss_D_monet_real = gan_loss(pred_real_monet, torch.ones_like(pred_real_monet))
            loss_D_monet_fake = gan_loss(pred_fake_monet, torch.zeros_like(pred_fake_monet))
            loss_D_photo_real = gan_loss(pred_real_photo, torch.ones_like(pred_real_photo))
            loss_D_photo_fake = gan_loss(pred_fake_photo, torch.zeros_like(pred_fake_photo))
            
            
            monet_D_loss = (loss_D_monet_real + loss_D_monet_fake) / 2
            photo_D_loss = (loss_D_photo_real + loss_D_photo_fake) / 2
            
            loss_D = monet_D_loss + photo_D_loss
            loss_D_avg += loss_D.item() / train_total_batch
            
                      
            #backward
            loss_D.backward()
            OptimD.step()

            train_bar.set_postfix(epoch = epoch+1, loss_G = loss_G.item(),loss_D = loss_D.item())
    
    schedulerD.step()
    schedulerG.step()
          
    wandb.log({'Epoch' : epoch+1, "Generator loss": loss_G_avg, 
               "Discriminator loss": loss_D_avg, 
               'Generator Learning rate' : OptimG.param_groups[0]['lr'], 
               'Discriminator Learning rate' : OptimD.param_groups[0]['lr']})
    
    if ((epoch+1) == 1) | ((epoch+1) % 1 == 0):
        
        with torch.no_grad():
            fake_photos = GM2P(monets)
            fake_monets = GP2M(photos)

            monets_ = monets[0,:,:,:]
            fake_photos = fake_photos[0,:,:,:]
            photos_ = photos[0,:,:,:]
            fake_monets = fake_monets[0,:,:,:]
            visulization(photos_, fake_monets)
        

        
        print('🔨Model Save🔨')
        torch.save(GM2P.state_dict(), './GM2P.pt')
        torch.save(GP2M.state_dict(), './GP2M.pt')
        torch.save(DM2P.state_dict(), './DM2P.pt')
        torch.save(DP2M.state_dict(), './DP2M.pt')
        
wandb.finish()

#### Did it work?
The dataset contains four directories: monet_tfrec, photo_tfrec, monet_jpg, and photo_jpg. The monet_tfrec and monet_jpg directories contain the same painting images, and the photo_tfrec and photo_jpg directories contain the same photos.
The monet directories contain Monet paintings. Use these images to train your model.

The photo directories contain photos. Add Monet-style to these images and submit your generated jpeg images as a zip file. Other photos outside of this dataset can be transformed but keep your submission file limited to 10,000 images.

#### What did you not understand about this process?
Well, everything provides in the competition data page. I've no problem while working on it. If you guys don't understand the thing that I'll do in this notebook then please comment on this notebook.

#### What else do you think you can try as part of this approach?
Monet-style art can be created from scratch using other GAN architectures like DCGAN.
Become more familiar with these concepts:

- Computer vision
- Generative models
- Tensor Processing Units (TPUs)
- TFRecords format for TensorFlow